First, we import all the necessary libraries

In [1]:
import copy
import mesa
import random
import matplotlib.pyplot as plt
import numpy as np
from mesa import Agent, Model
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid
from scipy.spatial.distance import cdist, squareform
from mesa.time import RandomActivation #mesa.time governs model time ticks.  The RandomActivation means that 
                                       #which agents go first in each time tick is randomly decided

We create a function that returns 0 or 1 which is the course of action assigned to the agent on initialization.

In [16]:
def course_of_action():
    """This function returns the course of action assigned to the agent
       on initialization.
       
    Returns
    -------
    action_course : list
        Action taken by the agent.
    """
    choices = [0, 1] #0 interception, 1 no interception
    # 80% interception and 20% no interception by the bad agent
    action_course = random.choices(choices, weights = [80, 20])[0]
    
    return action_course

In [54]:
class DiploModel(Model):
    
    """
    This class Initializes the model as an instance of the mesa.Model class.
    It include the following helper functions: __id_matrix, 
    """
    
    def __id_matrix(self):
        """This function calculates the inverse distance matrix of 1 over twice the distance.
        
        Returns
        -------
        ewise_dist : tuple
            Elementwise distance between two cells (array).
        """
        
        ## Gets the list of coordinates in the MultiGrid for the model and converts it into an array so we can 
        ## use the cdist function
        cell_list = list(self.grid.coord_iter())
        coords = [(cell[1], cell[2]) for cell in cell_list]
        
        ## cdist calculates the elementwise distance between two arrays - here we're just doing it between the array
        ## and itself, which gives us the distance matrices for each cell. Then convert the 0s to 1s to prevent 
        ## by zero, and take the inverse division
        ewise_dist = cdist(np_coords, np_coords)*2
        ewise_dist[ewise_dist == 0] = 1
        ewise_dist = (1 / ewise_dist)
        
        return ewise_dist
    
    def __valence_sum(self):
        """This function directly accesses the model's grid.coord_iter method and iterate over it. 
        
        Returns
        -------
        cell_valence : tuple
            Valence values for each cell.
        """
        cell_valence = copy.deepcopy(self.empty) # Instantiates IDW as a deepcopy of the 1d array of zeroes

        for cell in self.grid.coord_iter():
            
            ## Each cell in the grid has three attributes - a list of cell contents, the x and y coordinates,
            cell_content, x, y = cell 
            cell_valence[x][y] = sum([agent.valence for agent in cell_content])
            
        return(cell_valence)
    
    def __IDW_matrix(self):
        """This function calculates the inverse distance weights for each cell of the matrix.
        
        Returns
        -------
        IDW_results : tuple
            Inverse distance weighted values for that one layer in a flattened array.
        """
        IDW = copy.deepcopy(self.empty.flatten())
        
        #for every layer in the inverse distance cube, multiply the inverse distance and the
        ## agent counts together.  
        for i in range(len(self.id_matrix)):
            IDW_layer = np.multiply(self.id_matrix[i], self.valence_sum.flatten()[i])
            
            ## Add array elementwise
            IDW = np.add(IDW, IDW_layer) 
            
            ## Then, reshape it so it has the same dimensions as the model grid so we can extract neighborhood
            ## cell values, which will inform how the agent moves.
            IDW_results = (IDW.reshape(self.grid.width,self.grid.height)) 
            
        return IDW_results
    
    def __get_ids(self, a, b, g):
        """This function creates IDs for the three agents to enable the scheduler to know them.
        
        Parameters
        ----------
        a: interger
            ID for agent type a
        b: interger
            ID for agent type b
        g: interger
            ID for agent type g
        
        Returns
        -------
        a_ids, b_ids, g_ids : tuple
            a tuple containing ID lists of the three agents.     
        """
        a_ids = self.assign_list[0:a]
        b_ids = self.assign_list[a: (a + b)]
        g_ids = self.assign_list[(a + b):]
        
        return (a_ids, b_ids, g_ids)
    
    def __add_agent_list_to_schedule(self, id_list, agent_type):
        """This function adds all the created agents to the schedule in readiness for movement.

        Parameters
        ----------
        id_list: list
            List containing as a set of IDs for a given agent type
        agent_type: interger
            Agent type, can either be a(alpha), b(beta) or g(gamma)
        
        Returns
        -------
        agent_details : tuple
            Contains agent type and location in the grid.     
        """
        for i in id_list:            # Iterate over the agent's ID list
            a = agent_type(i, self)
            
            ## Adds the created agents to the schedule, so the scheduler knows they are there, and can move them
            ## at each time tick.
            self.schedule.add(a)
            
            ## Add the agent to a random grid cell at some random x and random y
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            
            ##  Can be read as place agent a at the coordinate tuple in the grid
            agent_details = self.grid.place_agent(a, (x, y))
            
        return agent_details

In [60]:
type((2, (4, 6)))

tuple